In [2]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'charlie'], address='local')
alice, bob, charlie = sf.PYU('alice'), sf.PYU('bob'), sf.PYU('charlie')


from secretflow.data.horizontal import read_csv
from secretflow.security.aggregation import SecureAggregator
from secretflow.security.compare import SPUComparator
from secretflow.utils.simulation.datasets import load_dermatology

aggr = SecureAggregator(charlie, [alice, bob])
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))
comp = SPUComparator(spu)
data = load_dermatology(parts=[alice, bob], aggregator=aggr, comparator=comp)
data.fillna(value=0, inplace=True)


The version of SecretFlow: 1.8.0b0


/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-27 18:55:13,569	INFO worker.py:1724 -- Started a local Ray instance.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.Actor_Masker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


In [4]:
sf.reveal(data.partitions)

{PYURuntime(alice): <secretflow.data.core.partition.Partition at 0x7f9c0e6c8550>,
 PYURuntime(bob): <secretflow.data.core.partition.Partition at 0x7f9c0e6cbc40>}

In [7]:

params = {
    # XGBoost parameter tutorial
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    'max_depth': 4,  # max depth
    'eta': 0.3,  # learning rate
    'objective': 'multi:softmax',  # objection function，support "binary:logistic","reg:logistic","multi:softmax","multi:softprob","reg:squarederror"
    'min_child_weight': 1,  # The minimum value of weight
    'lambda': 0.1,  # L2 regularization term on weights (xgb's lambda)
    'alpha': 0,  # L1 regularization term on weights (xgb's alpha)
    'max_bin': 10,  # Max num of binning
    'num_class': 6,  # Only required in multi-class classification
    'gamma': 0,  # Same to min_impurity_split,The minimux gain for a split
    'subsample': 1.0,  # Subsample rate by rows
    'colsample_bytree': 1.0,  # Feature selection rate by tree
    'colsample_bylevel': 1.0,  # Feature selection rate by level
    'eval_metric': 'merror',  # supported eval metric：
    # 1. rmse
    # 2. rmsle
    # 3. mape
    # 4. logloss
    # 5. error
    # 6. error@t
    # 7. merror
    # 8. mlogloss
    # 9. auc
    # 10. aucpr
    # Special params in SFXgboost
    # Required
    'hess_key': 'hess',  # Required, Mark hess columns, optionally choosing a column name that is not in the data set
    'grad_key': 'grad',  # Required，Mark grad columns, optionally choosing a column name that is not in the data set
    'label_key': 'class',  # Required，ark label columns, optionally choosing a column name that is not in the data set
}



from secretflow.ml.boost.homo_boost import SFXgboost

bst = SFXgboost(server=charlie, clients=[alice, bob])

bst.train(data, data, params=params, num_boost_round=6)

sf.shutdown()

print(bst)

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorHomoBooster'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorHomoBooster'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorHomoBooster'> with party charlie.
INFO:root:Create proxy actor <class 'abc.ActorHomoBinningBase'> with party alice.
INFO:root:Create proxy actor <class 'abc.ActorHomoBinningBase'> with party bob.
INFO:root:start recursive


(_run pid=3019196) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
(_run pid=3019196) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=3019196) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


(ActorHomoBooster pid=3020198) [0]	train-merror:0.02186	valid-merror:0.02186


(ActorHomoBooster pid=3020198) INFO:root:fit for iter_round=0 done
(_run pid=3019195) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
(_run pid=3019195) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=3019195) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


(ActorHomoBooster pid=3020199) [4]	train-merror:0.00546	valid-merror:0.00546 [repeated 14x across cluster]


(ActorHomoBooster pid=3020200) INFO:root:fit for iter_round=4 done [repeated 14x across cluster]
